In [11]:
import json
import numpy as np
import sqlalchemy
import pandas as pd
from fastapi import FastAPI, HTTPException
from langchain import LLMChain
from langchain.llms import Ollama
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate

# JSON Configuration and Metadata Data Embedded within the Script
db_config = {
    "dialect": "mysql",
    "username": "root",
    "password": "password",
    "host": "localhost",
    "port": 3306,
    "database": "otl"
}

metadata = {
    "tables": {
        "products": {
            "columns": {
                "id": "integer",
                "product_name": "string",
                "category_id": "integer",
                "price": "decimal",
                "created_at": "datetime"
            },
            "primary_key": ["id"],
            "description": "Contains information about products in the inventory."
        },
        "categories": {
            "columns": {
                "id": "integer",
                "category_name": "string",
                "description": "string"
            },
            "primary_key": ["id"],
            "description": "Contains product category information."
        },
        "sales": {
            "columns": {
                "id": "integer",
                "product_id": "integer",
                "quantity": "integer",
                "amount": "decimal",
                "sale_date": "datetime"
            },
            "primary_key": ["id"],
            "description": "Records each sale transaction with the product sold and the amount."
        },
        "imports": {
            "columns": {
                "id": "integer",
                "product_id": "integer",
                "quantity": "integer",
                "import_date": "datetime",
                "supplier": "string"
            },
            "primary_key": ["id"],
            "description": "Contains records of imported products."
        }
    },
    "relationships": {
        "products_sales": {
            "source_table": "sales",
            "target_table": "products",
            "source_column": "product_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each sale record references a product from the products table."
        },
        "products_categories": {
            "source_table": "products",
            "target_table": "categories",
            "source_column": "category_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each product belongs to a single category from the categories table."
        },
        "products_imports": {
            "source_table": "imports",
            "target_table": "products",
            "source_column": "product_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each import record references a product from the products table."
        }
    }
}

# Example Queries and Feedback Data
example_queries = {
    "queries": [
        # Add your example queries here
    ]
}

feedback_data = [
    # Add your feedback data here
]

# Metadata Retriever
class MetadataRetriever:
    def __init__(self, metadata):
        self.metadata = metadata
    
    def retrieve_metadata(self, query):
        # Retrieve relevant tables based on query
        relevant_tables = []
        for table in self.metadata["tables"]:
            if table in query:
                relevant_tables.append(table)
        return relevant_tables

# Llama Model for NL2SQL Conversion
class LlamaNL2SQL:
    def __init__(self, model_name="llama3:latest"):
        self.client = OllamaLLM(model=model_name)

    def query_to_sql(self, prompt):
        response = self.client.invoke(prompt)
        print("Model Response:", response)  # Debugging print to check response structure
        if isinstance(response, dict) and 'query' in response:
            return response['query']
        else:
            raise ValueError("Unexpected response format from LLM: " + str(response))
    
    def get_embedding(self, text):
        return np.random.rand(128)  # Example embedding

# RAG Query Chain
class RAGQueryChain:
    def __init__(self):
        self.retriever = MetadataRetriever(metadata)
        self.generator = LlamaNL2SQL()

    def get_sql_query(self, user_query):
        context_indices = self.retriever.retrieve_metadata(user_query)
        context = self.format_context(context_indices)
        sql_query = self.generator.query_to_sql(user_query + " " + context)
        return sql_query
    
    def format_context(self, context_indices):
        context = " ".join([str(index) for index in context_indices])
        return context

# Database Connector
class DBConnector:
    def __init__(self, config):
        self.engine = sqlalchemy.create_engine(
            f"{config['dialect']}://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
        )

    def execute_query(self, query):
        try:
            with self.engine.connect() as connection:
                result = pd.read_sql(query, connection)
            return result
        except Exception as e:
            return str(e)

# Error Handling
class ErrorHandler:
    def parse_error(self, error_message):
        if "syntax error" in error_message:
            return "Syntax Error", "Check the SQL syntax."
        elif "no such table" in error_message:
            return "Missing Table", "Check the table name in your query."
        return "Unknown Error", "An unknown error occurred."

    def suggest_fix(self, error_type, query):
        if error_type == "Syntax Error":
            return "Try rephrasing the query or correcting SQL syntax."
        elif error_type == "Missing Table":
            return f"Make sure the table exists in the database: {query}"
        return "Try modifying the query."

# Report Formatter
class ReportFormatter:
    def format_report(self, data):
        return data.to_string(index=False)

# Feedback Handler
class FeedbackHandler:
    def __init__(self, feedback_data):
        self.feedback_data = feedback_data

    def store_feedback(self, query, feedback, sql_query):
        self.feedback_data.append({"query": query, "feedback": feedback, "sql_query": sql_query})
        print("Feedback stored:", {"query": query, "feedback": feedback, "sql_query": sql_query})

# Report Generation Chain
class ReportChain:
    def __init__(self):
        self.rag_chain = RAGQueryChain()
        self.db_connector = DBConnector(config=db_config)
        self.error_handler = ErrorHandler()
        self.report_formatter = ReportFormatter()
        self.feedback_handler = FeedbackHandler(feedback_data)

    def generate_report(self, user_query):
        sql_query = self.rag_chain.get_sql_query(user_query)
        print("Generated SQL Query:", sql_query)  # Debugging print to check the generated SQL query
        result = self.db_connector.execute_query(sql_query)

        if isinstance(result, str):
            error_type, error_message = self.error_handler.parse_error(result)
            suggestion = self.error_handler.suggest_fix(error_type, sql_query)
            return {"error": error_message, "suggestion": suggestion}
        
        formatted_report = self.report_formatter.format_report(result)
        return {"report": formatted_report, "sql_query": sql_query}

    def store_feedback(self, query, feedback, sql_query):
        self.feedback_handler.store_feedback(query, feedback, sql_query)

# Report Chain Usage Example
report_chain = ReportChain()
prompt = "GIVE ME THE USER LIST"
response = report_chain.generate_report(prompt)
print(response)


Model Response: I'm happy to help! However, I need a bit more context. There are many types of user lists depending on the platform or system you're referring to.

Could you please specify what kind of user list you're looking for? For example:

1. A list of users in a particular social media platform (e.g., Facebook, Twitter)?
2. A list of users in an email marketing tool (e.g., Mailchimp, Constant Contact)?
3. A list of registered users in a website or application?
4. A list of subscribers to a newsletter or podcast?

Please let me know which type of user list you need, and I'll do my best to provide it for you!


ValueError: Unexpected response format from LLM: I'm happy to help! However, I need a bit more context. There are many types of user lists depending on the platform or system you're referring to.

Could you please specify what kind of user list you're looking for? For example:

1. A list of users in a particular social media platform (e.g., Facebook, Twitter)?
2. A list of users in an email marketing tool (e.g., Mailchimp, Constant Contact)?
3. A list of registered users in a website or application?
4. A list of subscribers to a newsletter or podcast?

Please let me know which type of user list you need, and I'll do my best to provide it for you!